# PARENT/AI-4-NICU Training School Hands-On Workshops

## Lab 3. AI for medical imaging predictions

1. Install Kaggle's CLI and download the dataset

In [ ]:
! pip install -q kaggle

In [ ]:
! kaggle datasets download sovitrath/diabetic-retinopathy-224x224-gaussian-filtered

Since our time on this worshop is quite limited we'll use a [preprocessed dataset](https://www.kaggle.com/datasets/sovitrath/diabetic-retinopathy-224x224-gaussian-filtered). The gaussian kernel was applied to every image in the [source dataset](https://www.kaggle.com/c/aptos2019-blindness-detection/overview) to reduce the images size.  

2. Unzip the dataset and move it to `dataset` directory for easier access

In [ ]:
!unzip -qq diabetic-retinopathy-224x224-gaussian-filtered

In [ ]:
!mkdir dataset
!mv gaussian_filtered_images/gaussian_filtered_images/* dataset/
!rm -rf gaussian_filtered_images

3. Import the dataset

In [ ]:
import pandas as pd

In [ ]:
labeled_data = pd.read_csv('train.csv')
labeled_data

4. The numerical labels are quite problematic, we don't want to learn them by heart, so let's convert them to text labels for easier interpretation

In [ ]:
DIAGNOSES = {
    0: 'No_DR',
    1: 'Mild',
    2: 'Moderate',
    3: 'Severe',
    4: 'Proliferate_DR',
}
labeled_data['label'] = labeled_data['diagnosis'].map(DIAGNOSES)

In [ ]:
labeled_data['label'].value_counts()

5. Our dataset contains a multilabel column. This means we should be able to train a model that can distinguish between different severities of the Diabetic Retinopaty. Let's add a binary label column in case we ever decided to train a binary classifier.

In [ ]:
labeled_data['binary_label'] = labeled_data['diagnosis'].map(lambda x: "DR" if x != 0 else "No_DR")

In [ ]:
labeled_data['binary_label'].value_counts()

6. Let's add the image file paths to our data, for convenient access in the future steps.

In [ ]:
labeled_data['filename'] = "dataset/" + labeled_data['label'] + "/" + labeled_data['id_code'] + ".png"

7. Let's see what exactly are we working with

In [ ]:
import cv2
import matplotlib.pyplot as plt

def display_image(path):
  img = cv2.imread(path)
  display(img)

In [ ]:
for image in labeled_data['filename'].head():
  display_image(image)

8. Training, validation, test division

In [ ]:
from sklearn.model_selection import train_test_split

train_test, validation = train_test_split(labeled_data, test_size = 0.15, stratify = labeled_data['label'], random_state=1)
train, test = train_test_split(train_test, test_size = 0.15 / (1 - 0.15), stratify = train_test['label'], random_state=1)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

def get_batches(df):
  return ImageDataGenerator(
      rescale = 1./255,
      # Data augmentation
      # zoom_range = 0.2,
      # rotation_range = 20,
      # shear_range = 0.2
    ).flow_from_dataframe(
      df,
      x_col="filename",
      y_col="binary_label",
      # y_col="label",
      target_size=(224, 224),
      class_mode='binary',
      shuffle=False,
    )

train_batches = get_batches(train)
validation_batches = get_batches(validation)
test_batches = ImageDataGenerator(rescale = 1./255).flow_from_dataframe(
    test,
    x_col="filename",
    y_col="binary_label",
    # y_col="label",
    target_size=(224, 224),
    shuffle=False,
    class_mode='binary'
)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential([
    layers.Conv2D(8, (3,3), padding="valid", input_shape=(224,224,3), activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    # layers.Dropout(),

    layers.Conv2D(16, (3,3), padding="valid", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    # layers.Dropout(),

    layers.Conv2D(32, (4,4), padding="valid", activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    # layers.Dropout(),

    layers.Flatten(),
    layers.Dense(32, activation = 'relu'),
    # layers.Dropout(0.15),
    layers.Dense(1, activation = 'sigmoid')
    # layers.Dense(5, activation = 'softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(),
    # loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['acc']
)

model.summary()

history = model.fit(
    train_batches,
    epochs=30,
    validation_data=validation_batches
)

To learn more about the layers used in this model see the documentation:
* [BatchNormalization](https://keras.io/api/layers/normalization_layers/batch_normalization/)
* [Dropout](https://keras.io/api/layers/regularization_layers/dropout/)

In [ ]:
model.evaluate(test_batches)

In [ ]:
import numpy as np

predictions = model.predict(test_batches)
# predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = np.rint(predictions.flatten())

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(test_batches.classes, predicted_labels)
ConfusionMatrixDisplay(cm, display_labels=test_batches.class_indices.keys()).plot()

### Transfer learning

In [ ]:
!pip install -q efficientnet

In [ ]:
import efficientnet.tfkeras as efn

transfer_model = tf.keras.Sequential([
  efn.EfficientNetB0(
      input_shape=(224, 224, 3),
      weights='imagenet',
      include_top=False
  ),
  tf.keras.layers.Flatten(),
  # tf.keras.layers.Dense(5, activation='softmax')
  tf.keras.layers.Dense(1, activation='sigmoid')
])

transfer_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    #loss=tf.keras.losses.CategoricalCrossentropy(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['acc']
)

history = transfer_model.fit(
    train_batches,
    epochs=30,
    validation_data=validation_batches
)


In [ ]:
binary_transfer = transfer_model

In [ ]:
transfer_model.evaluate(test_batches)

### Grid search

In [ ]:
LAYERS_CONFIGURATIONS = {
    "3 layer sets": (
        layers.Conv2D(16, (3,3), padding="valid", activation = 'relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.BatchNormalization(),

        layers.Conv2D(32, (4,4), padding="valid", activation = 'relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.BatchNormalization(),
    ),
    "2 layer sets": (
        layers.Conv2D(16, (3,3), padding="valid", activation = 'relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.BatchNormalization(),
    )
}

In [ ]:
LEARNING_RATES = [1e-4, 1e-5]

In [ ]:
LOSS_FUNCTIONS = ['categorical_crossentropy']

In [ ]:
def evaluate_model(layer_configuration, learning_rate, loss_function):
  model = tf.keras.Sequential([
    layers.Conv2D(8, (3,3), padding="valid", input_shape=(224,224,3), activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Dropout(0.5),

    *layer_configuration,

    layers.Flatten(),
    layers.Dense(32, activation = 'relu'),
    #layers.Dense(5, activation = 'softmax')
    layers.Dense(1, activation = 'sigmoid')
  ])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
      loss=loss_function,
      metrics=['acc']
  )

  model.fit(
      train_batches,
      epochs=30,
      validation_data=validation_batches
  )

  return model.evaluate(test_batches)[1]

In [ ]:
results = {}

for layer_configuration_name, layer_configuration in LAYERS_CONFIGURATIONS.items():
  for loss_function in LOSS_FUNCTIONS:
    for learning_rate in LEARNING_RATES:
      results[f"{layer_configuration_name} x {loss_function} x {learning_rate}"] = evaluate_model(
          layer_configuration=layer_configuration,
          learning_rate=learning_rate,
          loss_function=loss_function,
      )

In [ ]:
results

In [ ]:
max(results.items(), key=lambda entry: entry[1])

### Saving and loading the model

Since the learning process of the deep learning models could be very lengthy it may be a good idea to store them on the disk for future use, after they are properly trained.

In [ ]:
import pickle
pickle.dump(transfer_model, open("model.pickle", 'wb'))

In [ ]:
loaded_model = pickle.load(open("model.pickle", 'rb'))

In [ ]:
import numpy as np

predictions = loaded_model.predict(test_batches)
# predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = np.rint(predictions.flatten())

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(test_batches.classes, predicted_labels)
ConfusionMatrixDisplay(cm, display_labels=test_batches.class_indices.keys()).plot()